## 없는 패키지 설치
pip install selenium pandas pyperclip matplotlib webdriver_manager openpyxl

In [ ]:
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
chrome_options = webdriver.ChromeOptions()
# chromedriver 경로 입력
webdriver_service = Service('/Users/sumyeongkim/Desktop/chromedriver.exe')

## 전체 거래 스크롤하는 코드 
- 최대 지점까지 스크롤하고 최대 지점이 지정한 스크롤 수보다 적으면 멈춤(무한루프 방지)
- 스크롤 오류나면 그 시점까지 저장됨

In [ ]:
import time
import random
import pyperclip
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 웹 드라이버 초기화
driver = webdriver.Chrome()
url = 'https://kream.co.kr/products/59834'
driver.get(url)
data_list = []

try:
    # 로그인 버튼 클릭
    login_button = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div[1]/div/div[1]/div/ul/li[5]/a'))
    )
    login_button.click()
    time.sleep(1)

    # 이메일 주소 입력
    email_input = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div[3]/div/div/div[1]/div/input'))
    )
    
    user_id = '아이디'
    
    pyperclip.copy(user_id)
    email_input.click()
    email_input.send_keys(Keys.COMMAND, 'v')
    time.sleep(4)
    
    # 비밀번호 입력
    pw_input = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div[3]/div/div/div[2]/div/input'))
    )
    
    user_pw = '비밀번호'
    pyperclip.copy(user_pw)
    pw_input.click()
    pw_input.send_keys(Keys.COMMAND, 'v')
    time.sleep(4)
    
    # 로그인 버튼 클릭
    login_button = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div[3]/div/div/div[3]/a'))
    )
    login_button.click()
    time.sleep(2)
    
    # 체결 내역 클릭
    check_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="panel1"]/a'))
    )
    check_button.click()
    time.sleep(2)

    # 내부 스크롤 접근
    scroll_element = driver.find_element(By.CLASS_NAME, "price_body")
    
    last_height = driver.execute_script("return arguments[0].scrollHeight", scroll_element)
    no_new_content_count = 0
    max_no_new_content_count = 5

    for i in range(1, 1000):  # 최대 시도 횟수 설정
        if no_new_content_count >= max_no_new_content_count:
            print("Reached the end of the scrollable area or no new content loaded.")
            break

        # 랜덤한 스크롤 값 생성
        scroll_value = random.randint(2000, 3000)
        driver.execute_script("arguments[0].scrollTop += arguments[1];", scroll_element, scroll_value)
        time.sleep(random.uniform(4, 12))

        new_height = driver.execute_script("return arguments[0].scrollHeight", scroll_element)
        if new_height == last_height:
            no_new_content_count += 1
            print(f"No new content loaded: attempt {no_new_content_count}")
        else:
            no_new_content_count = 0
            last_height = new_height

    print("스크롤 작업 완료")

    # 데이터 크롤링 수행
    data_list = []
    for i in range(1, 50000):  # 충분히 큰 숫자로 설정
        try:
            xpath1 = f'//*[@id="panel1"]/div/div/div[2]/div[{i}]/div[1]/span'
            xpath2 = f'//*[@id="panel1"]/div/div/div[2]/div[{i}]/div[2]/span'
            xpath3 = f'//*[@id="panel1"]/div/div/div[2]/div[{i}]/div[3]/span'

            element1 = driver.find_element(By.XPATH, xpath1)
            data_list.append(element1.text)

            element2 = driver.find_element(By.XPATH, xpath2)
            data = element2.text
            data = ''.join(char for char in data if char.isdigit())
            data_list.append(data)

            element3 = driver.find_element(By.XPATH, xpath3)
            data_list.append(element3.text)

        except Exception as e:
            print(f"Error processing div {i}: {e}")
            break

    print("데이터 크롤링 완료")

    # 데이터 저장
    col_data = [data_list[i:i+3] for i in range(0, len(data_list), 3)]
    df = pd.DataFrame(col_data, columns=['사이즈', '가격', '거래일'])

    excel_filename = 'output1.xlsx'
    df.to_excel(excel_filename, index=False, engine='openpyxl')
    print(f"데이터를 {excel_filename} 파일로 저장했습니다.")

except Exception as e:
    print(f"An error occurred: {e}")
    if data_list:
        col_data = [data_list[i:i+3] for i in range(0, len(data_list), 3)]
        df = pd.DataFrame(col_data, columns=['사이즈', '가격', '거래일'])
        excel_filename = 'output_partial.xlsx'
        df.to_excel(excel_filename, index=False, engine='openpyxl')
        print(f"Partial data saved to {excel_filename}")

finally:
    print("크롤링 완료")
    driver.quit()

## 날짜 지정해서 스크롤하고 해당 부분 크롤링하기
- 시작 종료 시점 제대로 안적으면 저장 안됨! 

In [ ]:
import time
import random
import pyperclip
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime

# 날짜 형식 지정
date_format = "%y/%m/%d"
start_date = datetime.strptime("24/07/18", date_format)
end_date = datetime.strptime("24/07/21", date_format)

# 웹 드라이버 초기화
driver = webdriver.Chrome()
url = 'https://kream.co.kr/products/53213'
driver.get(url)
data_list = []

try:
    # 로그인 버튼 클릭
    login_button = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div[1]/div/div[1]/div/ul/li[5]/a'))
    )
    login_button.click()
    time.sleep(1)

    # 이메일 주소 입력
    email_input = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div[3]/div/div/div[1]/div/input'))
    )
    
    user_id = 'wwh145@naver.com'
    # driver.execute_script("arguments[0].value = arguments[1];", email_input, user_id)
    
    pyperclip.copy(user_id)
    email_input.click()
    email_input.send_keys(Keys.COMMAND, 'v')
    time.sleep(4)
    
    # 비밀번호 입력
    pw_input = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div[3]/div/div/div[2]/div/input'))
    )
    
    user_pw = 'rlacodhks12!'
    pyperclip.copy(user_pw)
    pw_input.click()
    pw_input.send_keys(Keys.COMMAND, 'v')
    time.sleep(4)
    
    # 로그인 버튼 클릭
    login_button = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div[3]/div/div/div[3]/a'))
    )
    login_button.click()
    time.sleep(2)
    
    # 체결 내역 클릭
    check_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="panel1"]/a'))
    )
    check_button.click()
    time.sleep(2)

    # 내부 스크롤 접근
    scroll_element = driver.find_element(By.CLASS_NAME, "price_body")
    
    last_height = driver.execute_script("return arguments[0].scrollHeight", scroll_element)
    no_new_content_count = 0
    max_no_new_content_count = 5

    # 스크롤 반복 횟수와 고정된 스크롤 값 설정
    max_attempts = 50000
    scroll_value = 2500  # 고정된 스크롤 값

    date_reached = False

    for i in range(1, max_attempts + 1):
        if no_new_content_count >= max_no_new_content_count or date_reached:
            print("Reached the end of the scrollable area or no new content loaded.")
            break

        driver.execute_script("arguments[0].scrollTop += arguments[1];", scroll_element, scroll_value)
        time.sleep(random.uniform(4, 12))

        new_height = driver.execute_script("return arguments[0].scrollHeight", scroll_element)
        if new_height == last_height:
            no_new_content_count += 1
            print(f"No new content loaded: attempt {no_new_content_count}")
        else:
            no_new_content_count = 0
            last_height = new_height

        # 각 스크롤 후 데이터 확인
        data_list = []
        for j in range(1, 500000):  # 충분히 큰 숫자로 설정
            try:
                xpath1 = f'//*[@id="panel1"]/div/div/div[2]/div[{j}]/div[1]/span'
                xpath2 = f'//*[@id="panel1"]/div/div/div[2]/div[{j}]/div[2]/span'
                xpath3 = f'//*[@id="panel1"]/div/div/div[2]/div[{j}]/div[3]/span'

                element1 = driver.find_element(By.XPATH, xpath1)
                size = element1.text

                element2 = driver.find_element(By.XPATH, xpath2)
                price = element2.text
                price = ''.join(char for char in price if char.isdigit())

                element3 = driver.find_element(By.XPATH, xpath3)
                date_str = element3.text
                transaction_date = datetime.strptime(date_str, date_format)

                if start_date <= transaction_date <= end_date:
                    data_list.append([size, price, date_str])
                else:
                    if transaction_date < start_date:
                        print("Reached dates earlier than the specified range.")
                        date_reached = True
                        break

            except Exception as e:
                print(f"Error processing div {j}: {e}")
                break

    print("스크롤 및 데이터 크롤링 작업 완료")

    df = pd.DataFrame(data_list, columns=['사이즈', '가격', '거래일'])

    excel_filename = 'output1.xlsx'
    df.to_excel(excel_filename, index=False, engine='openpyxl')
    print(f"데이터를 {excel_filename} 파일로 저장했습니다.")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    print("크롤링 완료")
    driver.quit()